# Extract Book Titles from The Posts

For every post id: get the actual version of the post, normalize it and search for book titles.

In [ ]:
import multiprocessing
import re
from collections import Counter
from functools import partial
from pathlib import Path

import pandas as pd
from tqdm.contrib.concurrent import process_map

from Functions import normalize_text, get_occurrences_for_post

NON_ALPHANUMERIC = re.compile(r'[\W_]+')
SUBREDDIT_NAME = 'books'
EXTRACTED_BOOKS_PATH = './data/books.csv'
IDS_PATH = './data/post_ids.csv'
STORED_POSTS_PATH = './data/stored_posts.csv'
FAUX_TITLES_PATH = Path('./data/faux_book_titles')
BOOK_COUNT_DF = './data/book_counts_raw.csv'

%load_ext autotime

### Prepare Book Titles

And compile them into a regex.

In [2]:
book_titles_df = pd.read_csv(EXTRACTED_BOOKS_PATH, index_col=0)
book_titles_df

,Title,Normalized
0,006 and a Half,006andahalf
1,01-01-00,010100
2,05:58,0558
3,08 the Planet of the Tortoise Driver Little Pr...,08theplanetofthetortoisedriverlittleprince
4,"10,000 ways to say I love you",10000waystosayiloveyou
...,...,...
110477,Zvezdy--kholodnye igrushki,zvezdykholodnyeigrushki
110478,Zvirahwe,zvirahwe
110479,The Z Was Zapped,zwaszapped
110480,Zweite Auflage im Altertum; kulturgeschichtlic...,zweiteauflageimaltertumkulturgeschichtlichestu...


time: 679 ms (started: 2022-10-26 14:26:28 +02:00)


In [3]:
title_dict = dict(zip(book_titles_df.Normalized, book_titles_df.Title))
if FAUX_TITLES_PATH.exists():
    faux_titles = {normalize_text(faux_title) for faux_title in FAUX_TITLES_PATH.read_text().splitlines(keepends=False)}
else:
    faux_titles = set()

all_normalized_titles = list(set(book_titles_df.Normalized) - faux_titles)
all_normalized_titles.sort(reverse=True, key=lambda x: len(x))
print(f'Found {len(all_normalized_titles):,} book titles.')

Found 110,258 book titles.
time: 151 ms (started: 2022-10-26 14:26:29 +02:00)


In [4]:
posts_df = pd.read_csv(STORED_POSTS_PATH, index_col=0)
posts_df.fillna('', inplace=True)
posts_df

/scratch/slurm_tmpdir/job_21150968/ipykernel_458967/3677812895.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  posts_df = pd.read_csv(STORED_POSTS_PATH, index_col=0)


,ID,Year,Post Text,Comment Text
0,7cexh,2008,funnyparodysffcovers,
1,72bqq,2008,pirateshortfictionfromfantasymagazineshimmerfr...,
2,6sr73,2008,reviewihatedavidsedaris,
3,6dpea,2008,sevendeadlywordsofbookreviewingnewyorktimesblog,
4,7mbbb,2008,2008discoveries,
...,...,...,...,...
289576,x3c279,2022,the2022hugoawardwinnerstobeannouncedonsunday4s...,
289577,tozgxo,2022,juststartedthevegetarianandiwishthehusbandallt...,
289578,xagcox,2022,neverjudgeabookbyitsauthorsfatherearlierthisye...,
289579,xe8nhv,2022,somebriefthoughtsonthewomenofthecoppercountrya...,


time: 1.41 s (started: 2022-10-26 14:26:29 +02:00)


In [5]:
posts = list(posts_df.sample(frac=1).itertuples(index=False, name=None))    # shuffle for better time estimate for the process bar

time: 363 ms (started: 2022-10-26 14:26:31 +02:00)


### Compile the Book Titles to RegEx

The longest book title is first, the shortest is last. The resulting RegEx always returns the longest non-overlapping matches. This way, we always match on the longest title, even if a shorter title is a substring of the long title. (e.g. Only match for 'Pride and Prejudice and Zombies' and not for 'Pride and Prejudice' in the string: 'My favourite book is "Pride and Prejudice and Zombies"')

In [6]:
title_regex = '|'.join(all_normalized_titles)
title_regex = re.compile(title_regex)

time: 4.2 s (started: 2022-10-26 14:26:31 +02:00)


### Search Posts for Book Titles

In [7]:
results = process_map(
    partial(get_occurrences_for_post, title_regex=title_regex),
    posts,
    max_workers=multiprocessing.cpu_count()//2,
    chunksize=250
)

  0%|          | 0/289581 [00:00<?, ?it/s]

time: 2h 42min 53s (started: 2022-10-26 14:26:35 +02:00)


In [8]:
year, post_occ, comment_occ = zip(*results)

per_year_posts = dict()
per_year_comments = dict()

for y in set(year):
    per_year_posts[y] = dict()
    per_year_comments[y] = dict()

for y, p, c in results:
    if p:
        per_year_posts[y] = Counter(per_year_posts[y]) + Counter(p)
    if c:
        per_year_comments[y] = Counter(per_year_posts[y]) + Counter(p)

time: 2min 19s (started: 2022-10-26 17:09:29 +02:00)


In [9]:
# print("Ranked from posts:", sorted(list(post_occ.items()), key=lambda x: x[1], reverse=True))
# print("Ranked from comments:", sorted(list(post_occ.items()), key=lambda x: x[1], reverse=True))
# print("Ranked from posts and comments:",
#       sorted(list((Counter(comment_occ) + Counter(post_occ))
#                   .items()), key=lambda x: x[1], reverse=True))

time: 391 µs (started: 2022-10-26 17:11:48 +02:00)


In [10]:
book_ranking = []
for y in set(year):
    book_ranking.extend([(y, title_dict[book_title], book_title, 'post', occurrences)
                    for book_title, occurrences in per_year_posts[y].items()])
    book_ranking.extend([(y, title_dict[book_title], book_title, 'comment', occurrences)
                         for book_title, occurrences in per_year_comments[y].items()])

time: 104 ms (started: 2022-10-26 17:11:48 +02:00)


In [11]:
book_ranking

[(2016, 'Infinite jest', 'infinitejest', 'post', 173),
 (2016, 'The Tain', 'tain', 'post', 1242),
 (2016,
  'The Adventures of Sherlock Holmes',
  'adventuresofsherlockholmes',
  'post',
  1),
 (2016, 'Cryptic', 'cryptic', 'post', 4),
 (2016, 'Necromance', 'necromance', 'post', 4),
 (2016, 'Radiance', 'radiance', 'post', 16),
 (2016, 'Shortcut', 'shortcut', 'post', 4),
 (2016, 'Stung', 'stung', 'post', 2),
 (2016, "The Handmaid's Tale", 'handmaidstale', 'post', 38),
 (2016, 'A Clockwork Orange', 'clockworkorange', 'post', 35),
 (2016, 'Fahrenheit 451', 'fahrenheit451', 'post', 42),
 (2016, 'The Omen', 'omen', 'post', 1132),
 (2016, 'The Heir', 'heir', 'post', 1962),
 (2016, 'Mistresses', 'mistresses', 'post', 3),
 (2016, 'Preaching', 'preaching', 'post', 10),
 (2016, 'Fading', 'fading', 'post', 7),
 (2016, 'The Outside World', 'outsideworld', 'post', 3),
 (2016, 'Punished!', 'punished', 'post', 9),
 (2016,
  'Harry potter and the cursed child',
  'harrypotterandthecursedchild',
  'post

time: 53.6 ms (started: 2022-10-26 17:11:48 +02:00)


In [12]:
merged_frame = pd.DataFrame(book_ranking, columns=['Year', 'Title', 'Normalized Title', 'Post or Comment', 'Occurrences'])
merged_frame.sort_values('Occurrences', inplace=True, ascending=False, ignore_index=False)
merged_frame.reset_index(drop='index', inplace=True)
merged_frame

,Year,Title,Normalized Title,Post or Comment,Occurrences
0,2021,The Heir,heir,post,4084
1,2020,The Heir,heir,post,3237
2,2022,The Heir,heir,post,3153
3,2022,The Heir,heir,comment,3151
4,2018,The Heir,heir,post,2221
...,...,...,...,...,...
83564,2021,All the President's Men,allthepresidentsmen,post,1
83565,2021,Show-Stopper!,showstopper,post,1
83566,2021,The Angel of the Crows,angelofthecrows,post,1
83567,2021,The Witness for the Dead,witnessforthedead,post,1


time: 1.16 s (started: 2022-10-26 17:11:48 +02:00)


In [13]:
merged_frame.to_csv(BOOK_COUNT_DF)

time: 229 ms (started: 2022-10-26 17:11:49 +02:00)
